### Importação de Bibliotecas

###### Realiza a importação de bibliotecas necessárias para o desenvolvimento dos testes do modelo PLN de intenção.

In [28]:
import nltk
import string 
import unittest
import unicodedata
from nltk.corpus import stopwords
import dateparser
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from datetime import datetime, timedelta
import re

nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('portuguese'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Inteli\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Inteli\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Configuração do Unittest

###### Realiza a configuração do Unittest para execução dos testes do modelo de PLN de intenção.

In [29]:
# Função para configurar o TestRunner para um teste específico
def roda_teste_especifico(clase_teste, nome_teste=None):
  loader = unittest.TestLoader()
  if nome_teste:
    # Carregar teste específico
    suite = loader.loadTestsFromName(f'{clase_teste.__name__}.{nome_teste}', clase_teste)
  else:
    # Carregar todos os testes da classe
    suite = loader.loadTestsFromTestCase(clase_teste)

  runner = unittest.TextTestRunner(verbosity=2)
  runner.run(suite)

### Pré-processamento 

###### Realiza o pré-processamento do texto de input do usuário, removendo caracteres especiais, espaços em branco e quebras de linha.

> ###### O código abaixo foi formulado pelo grupo e está sendo utilizado para a realização dos testes com o objetivo de saber se está funcionando corretamente. Ele é utilizado em `notebooks/intent/model`

In [30]:
def preprocess_text(text):
  text = text.lower()
  
  text = text.translate(str.maketrans('', '', string.punctuation))

  text = text.strip()

  filtered_words = filter(lambda word: word not in stop_words, text.split())

  text = " ".join(filtered_words)
  
  text = ''.join((c for c in unicodedata.normalize('NFD', text) if unicodedata.category(c) != 'Mn'))
  
  return text

### Extração de Parâmetros 

###### Realiza a extração de parâmetros do texto de input do usuário como tags, órgão regulador, datas e tipos de documentos.

> ###### O código abaixo foi formulado pelo grupo e está sendo utilizado para a realização dos testes com o objetivo de saber se está funcionando corretamente. Ele é utilizado em `notebooks/intent/model`

In [31]:
tag_mapping = {
  'administradores carteiras': 'Administradores de Carteiras',
  'agencias classificacao risco credito': 'Agências de Classificação de Risco de Crédito',
  'agentes fiduciarios': 'Agentes Fiduciários',
  'alerta': 'Alerta',
  'analistas valores mobiliarios': 'Analistas de Valores Mobiliários',
  'assessores investimento': 'Assessores de Investimento',
  'ato declaratorio': 'Ato Declaratório',
  'atuacao irregular': 'Atuação Irregular',
  'audiencia publica': 'Audiência Pública',
  'auditor independente': 'Auditor Independente',
  'bdr': 'BDR',
  'cadastro participantes regulados': 'Cadastro de Participantes Regulados',
  'clubes investimento': 'Clubes de Investimento',
  'companhia': 'Companhia',
  'comunicado mercado': 'Comunicado ao Mercado',
  'concurso premio': 'Concurso/Prêmio',
  'consultores valores mobiliarios': 'Consultores de Valores Mobiliários',
  'convenio': 'Convênio',
  'coronavirus': 'Coronavirus',
  'corretora': 'Corretora',
  'crowdfunding': 'Crowdfunding',
  'decisao colegiado': 'Decisão do Colegiado',
  'deliberacao': 'Deliberação',
  'educacao financeira': 'Educação Financeira',
  'evento': 'Evento',
  'fundos investimento': 'Fundos de Investimento',
  'fundos investimento direitos creditorios': 'Fundos de Investimento em Direitos Creditórios',
  'fundos investimento participacoes': 'Fundos de Investimento em Participações',
  'fundos investimento imobiliarios': 'Fundos de Investimento Imobiliários',
  'gestao institucional': 'Gestão Institucional',
  'indenizacao': 'Indenização',
  'infraestrutura mercado': 'Infraestrutura do Mercado',
  'insider trading': 'Insider Trading',
  'intermediarios': 'Intermediários',
  'investidores nao residentes': 'Investidores Nao Residentes',
  'julgamento': 'Julgamento',
  'julgamento insider': 'Julgamento_Insider',
  'mercados organizados': 'Mercados Organizados',
  'normas contabeis': 'Normas Contábeis',
  'nota': 'Nota',
  'ofertas publicas': 'Ofertas Publicas',
  'oficio circular': 'Ofício Circular',
  'ouvidoria': 'Ouvidoria',
  'parecer orientacao': 'Parecer de Orientação',
  'pesquisa': 'Pesquisa',
  'planejamento estrategico': 'Planejamento Estratégico',
  'pld ftp': 'PLD/FTP',
  'processo eletronico': 'Processo Eletrônico',
  'protocolo digital': 'Protocolo Digital',
  'publicacao': 'Publicação',
  'ritos cvm': 'Ritos CVM',
  'sandbox regulatorio': 'Sandbox Regulatório',
  'securitizadoras': 'Securitizadoras',
  'sistema governanca gestao cvm': 'Sistema de governança e gestão da CVM',
  'suitability': 'Suitability',
  'suspensao': 'Suspensão',
  'tecnologia informacao': 'Tecnologia da Informação',
  'termo compromisso': 'Termo de Compromisso',
  'termo compromisso insider': 'Termo_Compromisso_Insider'
}

In [32]:
def extract_dates(text):
    
  text = text.lower()
  text = text.replace('mes', 'mês').replace('ate', 'até')

  extracted_dates = []

  interval_match = re.search(r'de (\d{1,2}/\d{1,2}/\d{2,4}) até (\d{1,2}/\d{1,2}/\d{2,4})', text, re.IGNORECASE) or \
                    re.search(r'entre (\d{1,2}/\d{1,2}/\d{2,4}) e (\d{1,2}/\d{1,2}/\d{2,4})', text, re.IGNORECASE)
    
  if interval_match:
    start_date_str, end_date_str = interval_match.groups()
    start_date = dateparser.parse(start_date_str, languages=['pt'])
    end_date = dateparser.parse(end_date_str, languages=['pt'])
    extracted_dates.append((start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d')))
    
  else:
    date_matches = re.findall(
      r'\b(?:\d{1,2}/\d{1,2}/\d{2,4}|\d{1,2} de \w+ de \d{4}|hoje|ontem|anteontem|semana passada|mês passado|ano passado|há \d+ (?:dias|semanas|meses|anos))\b',
      text, re.IGNORECASE
    )

    for date_str in date_matches:
      parsed_date = dateparser.parse(date_str, languages=['pt'])

      if parsed_date:
          if "semana passada" in date_str.lower():
            start_of_week = parsed_date - timedelta(days=parsed_date.weekday() + 1)
            end_of_week = start_of_week + timedelta(days=6) 
            extracted_dates.append((start_of_week.strftime('%Y-%m-%d'), end_of_week.strftime('%Y-%m-%d')))
          
          elif "mês passado" in date_str.lower():
            start_of_month = parsed_date.replace(day=1)
            end_of_month = (parsed_date.replace(day=1) + timedelta(days=32)).replace(day=1) - timedelta(days=1)
            extracted_dates.append((start_of_month.strftime('%Y-%m-%d'), end_of_month.strftime('%Y-%m-%d')))
          
          elif "ano passado" in date_str.lower():
            start_of_year = parsed_date.replace(month=1, day=1)
            end_of_year = parsed_date.replace(month=12, day=31)
            extracted_dates.append((start_of_year.strftime('%Y-%m-%d'), end_of_year.strftime('%Y-%m-%d')))
          
          else:
            extracted_dates.append(parsed_date.strftime('%Y-%m-%d'))

  return extracted_dates if extracted_dates else None

In [33]:
def extract_parameters(utterance, processed_utterance):
  document_type_regex = r'(instrucoes|pareceres orientacao|deliberacoes|decisoes conjuntas|oficios circulares|leis decretos|notas explicativas)'

  tags_regex = r'(administradores carteiras|agencias classificacao risco credito|agentes fiduciarios|alerta|analistas valores mobiliarios|assessores investimento|ato declaratorio|atuacao irregular|audiencia publica|auditor independente|bdr|cadastro participantes regulados|clubes investimento|companhia|comunicado mercado|concurso premio|consultores valores mobiliarios|convenio|coronavirus|corretora|crowdfunding|decisao colegiado|deliberacao|educacao financeira|evento|fundos investimento|fundos investimento direitos creditorios|fundos investimento participacoes|fundos investimento imobiliarios|gestao institucional|indenizacao|infraestrutura mercado|insider trading|intermediarios|investidores nao residentes|julgamento|julgamento insider|mercados organizados|normas contabeis|nota|ofertas publicas|oficio circular|ouvidoria|parecer orientacao|pesquisa|planejamento estrategico|pld ftp|processo eletronico|protocolo digital|publicacao|ritos cvm|sandbox regulatorio|securitizadoras|sistema governanca gestao cvm|suitability|suspensao|tecnologia informacao|termo compromisso|termo compromisso insider)'

  entity_regex = r'cvm'

  document_type = re.search(document_type_regex, processed_utterance)
  tags = re.search(tags_regex, processed_utterance)
  entity = re.search(entity_regex, processed_utterance)
  dates = extract_dates(utterance)

  if tags:
    tags = tag_mapping.get(tags.group(0), tags.group(0))

  return {
    'document_type': document_type.group(0) if document_type else None,
    'tags': tags if tags else None,
    'entity': entity.group(0) if entity else None,
    'dates': dates if dates else None
  }

### Teste 1 - Verificação da Previsão de Intenção:

In [34]:
class TestePrevisaoDeIntencao(unittest.TestCase):

  # Configuração inicial
  def setUp(self):
    # Caminho para o arquivo JSON contendo os dados de intenções
    self.file_path = "../../../data/processed/intents.json"

    # Carrega dados 
    self.df = pd.read_json(self.file_path)

    # Processamento dos dados 
    self.df['processed_utterance'] = self.df['utterance'].apply(preprocess_text)

    # Treinamento do Modelo
    self.vectorizer = TfidfVectorizer()
    X = self.vectorizer.fit_transform(self.df['processed_utterance'])
    X_train, X_test, y_train, y_test = train_test_split(X, self.df['intent'], test_size=0.2, random_state=42)
    
    self.model = MultinomialNB()
    self.model.fit(X_train, y_train)
    self.X_test = X_test
    self.y_test = y_test
    self.y_pred = self.model.predict(X_test)

  # Teste da previsão do modelo
  def test_predict_intent(self):

    # Verificação da previsão de intenção
    frases_teste = [
      ("Quais são as instruções para esse processo?", "consulta_normativa"),
      ("Gostaria de saber sobre o prazo de validade.", "consulta_normativa"),
      ("Qual a melhor forma de investimento?", "consulta_normativa"),
      ("Quais são as últimas normas da CVM?", "consulta_normativa"),
      ("Qual a previsão do clima de hoje?", "outros"),
      ("Como fazer um bolo de chocolate?", "outros"),
    ]

    previsoes_corretas = 0

    for frase, intencao_esperada in frases_teste:
      frase_processada = preprocess_text(frase)
      frase_vetorizada = self.vectorizer.transform([frase_processada])
      intencao_prevista = self.model.predict(frase_vetorizada)[0]

      # Verifica se a intenção prevista corresponde à intenção esperada
      self.assertEqual(intencao_prevista, intencao_esperada, f"Frase '{frase}' foi prevista como '{intencao_prevista}' ao invés de '{intencao_esperada}'.")
      if intencao_prevista == intencao_esperada:
          previsoes_corretas += 1

    # Avaliar o desempenho do modelo para a classe 'consulta_normativa'
    precisao = precision_score(self.y_test, self.y_pred, pos_label="consulta_normativa", average='binary')
    recall = recall_score(self.y_test, self.y_pred, pos_label="consulta_normativa", average='binary')
    f1 = f1_score(self.y_test, self.y_pred, pos_label="consulta_normativa", average='binary')

    print(f"Precisão: {precisao:.2f}, Recall: {recall:.2f}, F1-score: {f1:.2f}")

    # Verifica se as métricas de desempenho estão acima do limite aceitável
    self.assertGreaterEqual(precisao, 0.8, "A precisão para 'consulta_normativa' está abaixo do esperado.")
    self.assertGreaterEqual(recall, 0.8, "A revocação para 'consulta_normativa' está abaixo do esperado.")
    self.assertGreaterEqual(f1, 0.8, "O F1-score para 'consulta_normativa' está abaixo do esperado.")

roda_teste_especifico(TestePrevisaoDeIntencao)

test_predict_intent (__main__.TestePrevisaoDeIntencao.test_predict_intent) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.039s

OK


Precisão: 1.00, Recall: 1.00, F1-score: 1.00


### Teste 2 - Extração de Parâmetros para "Consulta Normativa": 

In [35]:
class TestExtracaoParametrosConsultaNormativa(unittest.TestCase):

    # Configuração inicial
    def setUp(self):
        self.test_cases = [
            {
                "frase": "Quero ver as instruções sobre atuacao irregular",
                "esperado": {
                    "document_type": "instrucoes",
                    "tags": "Atuação Irregular",
                    "entity": None
                }
            },
            {
                "frase": "Preciso de informações sobre decisões do concurso recentes",
                "esperado": {
                    "document_type": None,
                    "tags": None,
                    "entity": None
                }
            },
            {
                "frase": "Quais são as diretrizes para auditor independente?",
                "esperado": {
                    "document_type": None,
                    "tags": "Auditor Independente",
                    "entity": None
                }
            },
            {
                "frase": "Informações sobre o planejamento estratégico da CVM são necessárias",
                "esperado": {
                    "document_type": None,
                    "tags": "Planejamento Estratégico",
                    "entity": "cvm"
                }
            },
            {
                "frase": "Quais são as regras de sandbox regulatório atualmente?",
                "esperado": {
                    "document_type": None,
                    "tags": "Sandbox Regulatório",
                    "entity": None
                }
            }
        ]

    # Teste de extração de parâmetros
    def test_extract_parameters(self):
        for case in self.test_cases:
            frase = case["frase"]
            esperado = case["esperado"]

            try:
                # Pré-processamento da frase antes de passar para a extração de parâmetros
                processed_frase = preprocess_text(frase)
                
                # Executar a função de extração de parâmetros
                parametros_extraidos = extract_parameters("", processed_frase)

                # Comparar os parâmetros extraídos com os valores esperados
                self.assertEqual(parametros_extraidos['document_type'], esperado['document_type'],
                                f"Erro na extração de 'document_type' para a frase: '{frase}'")
                self.assertEqual(parametros_extraidos['tags'], esperado['tags'],
                                f"Erro na extração de 'tags' para a frase: '{frase}'")
                self.assertEqual(parametros_extraidos['entity'], esperado['entity'],
                                f"Erro na extração de 'entity' para a frase: '{frase}'")
                
                # Printa a frase e os parâmetros extraídos
                print(f"Frase: '{frase}'")
                print(f"Parâmetros extraídos: {parametros_extraidos}")
                print("")

            except AssertionError as e:
                print(f"Falha no teste para a frase: '{frase}'. {str(e)}")
                raise e
            except Exception as e:
                print(f"Erro inesperado ao testar a frase: '{frase}'. {str(e)}")
                raise e

roda_teste_especifico(TestExtracaoParametrosConsultaNormativa)

test_extract_parameters (__main__.TestExtracaoParametrosConsultaNormativa.test_extract_parameters) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.005s

OK


Frase: 'Quero ver as instruções sobre atuacao irregular'
Parâmetros extraídos: {'document_type': 'instrucoes', 'tags': 'Atuação Irregular', 'entity': None, 'dates': None}

Frase: 'Preciso de informações sobre decisões do concurso recentes'
Parâmetros extraídos: {'document_type': None, 'tags': None, 'entity': None, 'dates': None}

Frase: 'Quais são as diretrizes para auditor independente?'
Parâmetros extraídos: {'document_type': None, 'tags': 'Auditor Independente', 'entity': None, 'dates': None}

Frase: 'Informações sobre o planejamento estratégico da CVM são necessárias'
Parâmetros extraídos: {'document_type': None, 'tags': 'Planejamento Estratégico', 'entity': 'cvm', 'dates': None}

Frase: 'Quais são as regras de sandbox regulatório atualmente?'
Parâmetros extraídos: {'document_type': None, 'tags': 'Sandbox Regulatório', 'entity': None, 'dates': None}



### Teste 3 - Processamento de Texto

In [36]:
class TestPreprocessamentoTexto(unittest.TestCase):

    # Configuração inicial
    def setUp(self):
      self.frases_teste = [
        {
          "frase": "Instruções da CVM sobre ESG!",
          "esperado": "instrucoes cvm sobre esg"
        },
        {
          "frase": "Quais são as regras de governança corporativa?",
          "esperado": "quais regras governanca corporativa"
        },
        {
          "frase": "Leia o relatório de sustentabilidade.",
          "esperado": "leia relatorio sustentabilidade"
        },
        {
          "frase": "As normas da CVM são obrigatórias?",
          "esperado": "normas cvm obrigatorias"
        },
        {
          "frase": "Pareceres de orientação sobre auditoria independente.",
          "esperado": "pareceres orientacao sobre auditoria independente"
        }
      ]

    # Teste de preprocessamento de texto
    def test_preprocess_text(self):
      for teste in self.frases_teste:
        frase = teste["frase"]
        esperado = teste["esperado"]

        try:
          # Executar a função de preprocessamento
          resultado = preprocess_text(frase)
          print(f"Frase sem pré-processamento: {frase}")
          print(f"Frase após pré-processamento: {resultado}")
          print("")

          # Verificar se o resultado corresponde ao esperado
          self.assertEqual(resultado, esperado, f"Erro no preprocessamento para a frase: '{frase}'")

        except AssertionError as e:
          print(f"Falha no teste para a frase: '{frase}'. {str(e)}")
          raise e
        except Exception as e:
          print(f"Erro inesperado ao testar a frase: '{frase}'. {str(e)}")
          raise e

roda_teste_especifico(TestPreprocessamentoTexto)


test_preprocess_text (__main__.TestPreprocessamentoTexto.test_preprocess_text) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


Frase sem pré-processamento: Instruções da CVM sobre ESG!
Frase após pré-processamento: instrucoes cvm sobre esg

Frase sem pré-processamento: Quais são as regras de governança corporativa?
Frase após pré-processamento: quais regras governanca corporativa

Frase sem pré-processamento: Leia o relatório de sustentabilidade.
Frase após pré-processamento: leia relatorio sustentabilidade

Frase sem pré-processamento: As normas da CVM são obrigatórias?
Frase após pré-processamento: normas cvm obrigatorias

Frase sem pré-processamento: Pareceres de orientação sobre auditoria independente.
Frase após pré-processamento: pareceres orientacao sobre auditoria independente



### Teste 4 - Validação do Modelo com Dados de Teste:

In [37]:
class TestValidacaoModeloComDadosDeTeste(unittest.TestCase):

    # Configuração inicial
    def setUp(self):
        self.file_path = "../../../data/processed/intents.json"
        
        # Carregar os dados
        self.df = pd.read_json(self.file_path)
        
        # Preprocessamento de dados
        self.df['processed_utterance'] = self.df['utterance'].apply(preprocess_text)
        
        # Vetorização dos dados
        self.vectorizer = TfidfVectorizer()
        X = self.vectorizer.fit_transform(self.df['processed_utterance'])
        X_train, X_test, y_train, y_test = train_test_split(X, self.df['intent'], test_size=0.2, random_state=42)
        
        # Treinamento do modelo
        self.model = MultinomialNB()
        self.model.fit(X_train, y_train)
        
        # Dados de teste
        self.X_test = X_test
        self.y_test = y_test
        self.y_pred = self.model.predict(X_test)

    def teste_performance_modelo(self):
        try:
            accuracy = accuracy_score(self.y_test, self.y_pred)
            print(f"Acurácia do modelo: {accuracy:.2%}")
            
            report = classification_report(self.y_test, self.y_pred)
            print("Relatório de Classificação:")
            print(report)

            # Verifica se a acurácia está acima de 80%
            self.assertGreaterEqual(accuracy, 0.8, "A acurácia do modelo está abaixo do esperado (80%).")

        except AssertionError as e:
            print(f"Falha no teste de validação do modelo. {str(e)}")
            raise e
        except Exception as e:
            print(f"Erro inesperado ao validar o modelo. {str(e)}")
            raise e

roda_teste_especifico(TestValidacaoModeloComDadosDeTeste)

teste_performance_modelo (__main__.TestValidacaoModeloComDadosDeTeste.teste_performance_modelo) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.036s

OK


Acurácia do modelo: 100.00%
Relatório de Classificação:
                    precision    recall  f1-score   support

consulta_normativa       1.00      1.00      1.00        21
            outros       1.00      1.00      1.00        19

          accuracy                           1.00        40
         macro avg       1.00      1.00      1.00        40
      weighted avg       1.00      1.00      1.00        40



### Teste 5 - Teste de Robustez para Frases Fora do Escopo:

In [38]:
class TestRobustezFrasesForaDoEscopo(unittest.TestCase):

    # Configuração inicial
    def setUp(self):
        # Configuração inicial com exemplos de frases fora do escopo e o resultado esperado
        self.frases_teste = [
            {
                "frase": "Qual é a previsão do tempo para amanhã?",
                "esperado": "outros"
            },
            {
                "frase": "Como faço para cozinhar arroz?",
                "esperado": "outros"
            },
            {
                "frase": "Quem ganhou o jogo de ontem?",
                "esperado": "outros"
            },
            {
                "frase": "Me conte uma piada engraçada!",
                "esperado": "outros"
            },
            {
                "frase": "Quais são as novidades no mundo da música?",
                "esperado": "outros"
            }
        ]

        # Configuração do modelo e vetor TF-IDF treinado
        self.file_path = "../../../data/processed/intents.json"
        
        # Carregar os dados de treinamento
        self.df = pd.read_json(self.file_path)
        self.df['processed_utterance'] = self.df['utterance'].apply(preprocess_text)

        # Treinamento do vetor TF-IDF com os dados originais
        self.vectorizer = TfidfVectorizer()
        X = self.vectorizer.fit_transform(self.df['processed_utterance'])
        
        # Treinamento do modelo
        self.model = MultinomialNB()
        X_train, X_test, y_train, y_test = train_test_split(X, self.df['intent'], test_size=0.2, random_state=42)
        self.model.fit(X_train, y_train)

    def test_robustez_frases_fora_do_escopo(self):
        for teste in self.frases_teste:
            frase = teste["frase"]
            esperado = teste["esperado"]

            try:
                # Pré-processamento da frase
                processed_frase = preprocess_text(frase)

                # Vetorizar a frase usando o vetor ajustado previamente
                vectorized_frase = self.vectorizer.transform([processed_frase])
                
                # Fazer a previsão de intenção
                predicted_intent = self.model.predict(vectorized_frase)[0]

                # Verificar se a intenção prevista é "outros"
                self.assertEqual(predicted_intent, esperado,
                                 f"Erro na classificação de frase fora do escopo: '{frase}' foi classificada como '{predicted_intent}' ao invés de '{esperado}'")

                # Verificar se o sistema não tenta extrair parâmetros de frases classificadas como "outros"
                if predicted_intent == "outros":
                    parametros_extraidos = extract_parameters("", processed_frase)
                    self.assertIsNone(parametros_extraidos['document_type'],
                                      f"Erro: Parâmetros indevidamente extraídos para a frase fora do escopo: '{frase}'")
                    self.assertIsNone(parametros_extraidos['tags'],
                                      f"Erro: Parâmetros indevidamente extraídos para a frase fora do escopo: '{frase}'")
                    self.assertIsNone(parametros_extraidos['entity'],
                                      f"Erro: Parâmetros indevidamente extraídos para a frase fora do escopo: '{frase}'")
                    
                # Printa a frase e a intenção prevista
                print(f"Frase: '{frase}'")
                print(f"Intenção prevista: {predicted_intent}")
                print("")

            except AssertionError as e:
                print(f"Falha no teste para a frase: '{frase}'. {str(e)}")
                raise e
            except Exception as e:
                print(f"Erro inesperado ao testar a frase: '{frase}'. {str(e)}")
                raise e

roda_teste_especifico(TestRobustezFrasesForaDoEscopo)


test_robustez_frases_fora_do_escopo (__main__.TestRobustezFrasesForaDoEscopo.test_robustez_frases_fora_do_escopo) ... ok


Frase: 'Qual é a previsão do tempo para amanhã?'
Intenção prevista: outros

Frase: 'Como faço para cozinhar arroz?'
Intenção prevista: outros

Frase: 'Quem ganhou o jogo de ontem?'
Intenção prevista: outros

Frase: 'Me conte uma piada engraçada!'
Intenção prevista: outros

Frase: 'Quais são as novidades no mundo da música?'
Intenção prevista: outros




----------------------------------------------------------------------
Ran 1 test in 0.033s

OK


### Teste 6 - Integração Completa com Previsão e Extração:

In [39]:
class TestIntegracaoCompletaPrevisaoExtracao(unittest.TestCase):

    # Configuração inicial
    def setUp(self):
        self.frases_teste = [
            {
                "frase": "Quais são as instruções sobre ESG?",
                "intencao_esperada": "consulta_normativa",
                "parametros_esperados": {
                    "document_type": "instrucoes",
                    "tags": None,
                    "entity": None
                }
            },
            {
                "frase": "Preciso de detalhes sobre a Resolução CVM referente a alerta",
                "intencao_esperada": "consulta_normativa",
                "parametros_esperados": {
                    "document_type": None,
                    "tags": "Alerta",
                    "entity": "cvm"
                }
            }
        ]

        self.file_path = "../../../data/processed/intents.json"
        
        # Carregar os dados de treinamento
        self.df = pd.read_json(self.file_path)
        self.df['processed_utterance'] = self.df['utterance'].apply(preprocess_text)

        # Treinamento do vetor TF-IDF com os dados originais
        self.vectorizer = TfidfVectorizer()
        X = self.vectorizer.fit_transform(self.df['processed_utterance'])
        
        # Treinamento do modelo
        self.model = MultinomialNB()
        X_train, X_test, y_train, y_test = train_test_split(X, self.df['intent'], test_size=0.2, random_state=42)
        self.model.fit(X_train, y_train)

    # Teste de integração completo
    def teste_integracao_completa_previsao_extracao(self):
        for teste in self.frases_teste:
            frase = teste["frase"]
            intencao_esperada = teste["intencao_esperada"]
            parametros_esperados = teste["parametros_esperados"]

            try:
                # Pré-processamento da frase
                processed_frase = preprocess_text(frase)

                # Vetorizar a frase usando o vetor ajustado previamente
                vectorized_frase = self.vectorizer.transform([processed_frase])
                
                # Fazer a previsão de intenção
                predicted_intent = self.model.predict(vectorized_frase)[0]

                # Verificar se a intenção prevista corresponde à intenção esperada
                self.assertEqual(predicted_intent, intencao_esperada,
                                 f"Erro na previsão de intenção: '{frase}' foi classificada como '{predicted_intent}' ao invés de '{intencao_esperada}'")

                # Se a intenção for "consulta_normativa", verificar a extração de parâmetros
                if predicted_intent == "consulta_normativa":
                    parametros_extraidos = extract_parameters("", processed_frase)
                    self.assertEqual(parametros_extraidos['document_type'], parametros_esperados['document_type'],
                                     f"Erro na extração de 'document_type' para a frase: '{frase}'")
                    self.assertEqual(parametros_extraidos['tags'], parametros_esperados['tags'],
                                     f"Erro na extração de 'tags' para a frase: '{frase}'")
                    self.assertEqual(parametros_extraidos['entity'], parametros_esperados['entity'],
                                     f"Erro na extração de 'entity' para a frase: '{frase}'")
                    
                # Printa a frase, a intenção prevista e os parâmetros extraídos
                print(f"Frase: '{frase}'")
                print(f"Intenção prevista: {predicted_intent}")
                print(f"Parâmetros extraídos: {parametros_extraidos}")
                print("")

            except AssertionError as e:
                print(f"Falha no teste para a frase: '{frase}'. {str(e)}")
                raise e
            except Exception as e:
                print(f"Erro inesperado ao testar a frase: '{frase}'. {str(e)}")
                raise e

roda_teste_especifico(TestIntegracaoCompletaPrevisaoExtracao)


teste_integracao_completa_previsao_extracao (__main__.TestIntegracaoCompletaPrevisaoExtracao.teste_integracao_completa_previsao_extracao) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.030s

OK


Frase: 'Quais são as instruções sobre ESG?'
Intenção prevista: consulta_normativa
Parâmetros extraídos: {'document_type': 'instrucoes', 'tags': None, 'entity': None, 'dates': None}

Frase: 'Preciso de detalhes sobre a Resolução CVM referente a alerta'
Intenção prevista: consulta_normativa
Parâmetros extraídos: {'document_type': None, 'tags': 'Alerta', 'entity': 'cvm', 'dates': None}



### Teste 7 - Extração de Intervalo de Datas Explícitas:

In [40]:
class TesteExtracaoIntervaloDatasExplicitas(unittest.TestCase):

    # Configuração inicial
    def setUp(self):
        self.casos_teste = [
            {
                "frase": "Quero ver os relatórios de 01/01/2022 até 31/01/2022",
                "datas_esperadas": [('2022-01-01', '2022-01-31')]
            },
            {
                "frase": "Preciso de informações entre 10/03/2021 e 20/03/2021 sobre os investimentos",
                "datas_esperadas": [('2021-03-10', '2021-03-20')]
            },
            {
                "frase": "Quais foram os dados de 05/05/2020 ate 15/05/2020?",
                "datas_esperadas": [('2020-05-05', '2020-05-15')]
            },
            {
                "frase": "Verifique as estatísticas de ontem",
                "datas_esperadas": ['2024-08-29']
            },
            {
                "frase": "Gostaria de ver os registros de hoje",
                "datas_esperadas": ['2024-08-30']
            },
            {
                "frase": "Quais sãos as normas da CVM do dia 20 de janeiro de 2023?",
                "datas_esperadas": ['2023-01-20']
            },

        ]

    # Teste de extração de intervalo de datas
    def teste_extracao_data(self):
        for teste in self.casos_teste:
            frase = teste["frase"]
            datas_esperadas = teste["datas_esperadas"]

            try:
                # Executar a função de extração de datas
                print(f"\nTestando frase: '{frase}'")
                datas_extraidas = extract_dates(frase)

                # Se datas_extraidas é None, fornecer mais detalhes para depuração
                if datas_extraidas is None:
                    print(f"Erro: Nenhuma data extraída para a frase: '{frase}'. Verifique o regex na função extract_dates.")
                else:
                    print(f"Datas extraídas: {datas_extraidas}")

                # Verificar se as datas extraídas correspondem às datas esperadas
                self.assertEqual(datas_extraidas, datas_esperadas,
                                 f"Erro na extração de datas para a frase: '{frase}'. Esperado: {datas_esperadas}, Encontrado: {datas_extraidas}")

            except AssertionError as e:
                print(f"Falha no teste para a frase: '{frase}'. {str(e)}")
                raise e
            except Exception as e:
                print(f"Erro inesperado ao testar a frase: '{frase}'. {str(e)}")
                raise e

roda_teste_especifico(TesteExtracaoIntervaloDatasExplicitas)


teste_extracao_data (__main__.TesteExtracaoIntervaloDatasExplicitas.teste_extracao_data) ... ok



Testando frase: 'Quero ver os relatórios de 01/01/2022 até 31/01/2022'
Datas extraídas: [('2022-01-01', '2022-01-31')]

Testando frase: 'Preciso de informações entre 10/03/2021 e 20/03/2021 sobre os investimentos'
Datas extraídas: [('2021-03-10', '2021-03-20')]

Testando frase: 'Quais foram os dados de 05/05/2020 ate 15/05/2020?'
Datas extraídas: [('2020-05-05', '2020-05-15')]

Testando frase: 'Verifique as estatísticas de ontem'
Datas extraídas: ['2024-08-29']

Testando frase: 'Gostaria de ver os registros de hoje'
Datas extraídas: ['2024-08-30']

Testando frase: 'Quais sãos as normas da CVM do dia 20 de janeiro de 2023?'
Datas extraídas: ['2023-01-20']



----------------------------------------------------------------------
Ran 1 test in 0.025s

OK


### Teste 8 - Extração de Referências Temporais Relativas:

In [41]:
class TesteExtracaoReferenciasTemporaisRelativas(unittest.TestCase):

  # Configuração inicial
  def setUp(self):
      # Data atual para referência nos cálculos de datas relativas
      self.data_atual = datetime.now().date()

      self.casos_teste = [
        {
          "frase": "Quero os dados da semana passada",
          "datas_esperadas": [
              (self.data_atual - timedelta(days=self.data_atual.weekday() + 8)).strftime('%Y-%m-%d'),
              (self.data_atual - timedelta(days=self.data_atual.weekday() + 2)).strftime('%Y-%m-%d')
          ]
        },
        {
          "frase": "Preciso de informações do mês passado",
          "datas_esperadas": [
              (self.data_atual.replace(day=1) - timedelta(days=1)).replace(day=1).strftime('%Y-%m-%d'),
              (self.data_atual.replace(day=1) - timedelta(days=1)).strftime('%Y-%m-%d')
          ]
        },
        {
          "frase": "Quais foram os dados do ano passado?",
          "datas_esperadas": [
              self.data_atual.replace(year=self.data_atual.year - 1, month=1, day=1).strftime('%Y-%m-%d'),
              self.data_atual.replace(year=self.data_atual.year - 1, month=12, day=31).strftime('%Y-%m-%d')
          ]
        }
      ]

  # Teste de extração de referências temporais relativas
  def teste_extracao_referencias_temporais_relativas(self):
      for teste in self.casos_teste:
        frase = teste["frase"]
        datas_esperadas = teste["datas_esperadas"]

        try:
          # Executar a função de extração de datas
          print(f"\nTestando frase: '{frase}'")
          datas_extraidas = extract_dates(frase)

          # Se datas_extraidas é None, fornecer mais detalhes para depuração
          if datas_extraidas is None:
            print(f"Erro: Nenhuma data extraída para a frase: '{frase}'. Verifique o regex na função extract_dates.")
          else:
            print(f"Datas extraídas: {datas_extraidas}")

          # Verificar se as datas extraídas correspondem às datas esperadas
          self.assertEqual(datas_extraidas, [tuple(datas_esperadas)],
                            f"Erro na extração de datas para a frase: '{frase}'. Esperado: {datas_esperadas}, Encontrado: {datas_extraidas}")

        except AssertionError as e:
          print(f"Falha no teste para a frase: '{frase}'. {str(e)}")
          raise e
        except Exception as e:
          print(f"Erro inesperado ao testar a frase: '{frase}'. {str(e)}")
          raise e

roda_teste_especifico(TesteExtracaoReferenciasTemporaisRelativas)


teste_extracao_referencias_temporais_relativas (__main__.TesteExtracaoReferenciasTemporaisRelativas.teste_extracao_referencias_temporais_relativas) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.011s

OK



Testando frase: 'Quero os dados da semana passada'
Datas extraídas: [('2024-08-18', '2024-08-24')]

Testando frase: 'Preciso de informações do mês passado'
Datas extraídas: [('2024-07-01', '2024-07-31')]

Testando frase: 'Quais foram os dados do ano passado?'
Datas extraídas: [('2023-01-01', '2023-12-31')]


### Teste 9 - Extração de Datas Simples e Padrões Diferentes:

In [42]:
class TesteExtracaoDatasSimplesFormatosDiferentes(unittest.TestCase):

    # Configuração inicial
    def setUp(self):
        self.casos_teste = [
            {
                "frase": "Preciso dos documentos de 10 de outubro de 2022",
                "datas_esperadas": ['2022-10-10']
            },
            {
                "frase": "Verifique os registros de 05/12/2021",
                "datas_esperadas": ['2021-12-05']
            },
            {
                "frase": "Quero informações das normas do ano passado",
                "datas_esperadas": [('2023-01-01', '2023-12-31')]
            },
            {
                "frase": "Veja os dados de 15/07/19",
                "datas_esperadas": ['2019-07-15']
            },
            {
                "frase": "Os relatórios lançados há 5 dias",
                "datas_esperadas": ['2024-08-25']
            }
        ]

    # Teste de extração de datas simples em formatos diferentes
    def teste_extracao_datas_simples_formatos_diferentes(self):
        for teste in self.casos_teste:
            frase = teste["frase"]
            datas_esperadas = teste["datas_esperadas"]

            try:
                # Executar a função de extração de datas
                print(f"\nTestando frase: '{frase}'")
                datas_extraidas = extract_dates(frase)

                # Se datas_extraidas é None, fornecer mais detalhes para depuração
                if datas_extraidas is None:
                    print(f"Erro: Nenhuma data extraída para a frase: '{frase}'. Verifique o regex na função extract_dates.")
                else:
                    print(f"Datas extraídas: {datas_extraidas}")

                # Verificar se as datas extraídas correspondem às datas esperadas
                self.assertEqual(datas_extraidas, datas_esperadas,
                                 f"Erro na extração de datas para a frase: '{frase}'. Esperado: {datas_esperadas}, Encontrado: {datas_extraidas}")

            except AssertionError as e:
                print(f"Falha no teste para a frase: '{frase}'. {str(e)}")
                raise e
            except Exception as e:
                print(f"Erro inesperado ao testar a frase: '{frase}'. {str(e)}")
                raise e

roda_teste_especifico(TesteExtracaoDatasSimplesFormatosDiferentes)


teste_extracao_datas_simples_formatos_diferentes (__main__.TesteExtracaoDatasSimplesFormatosDiferentes.teste_extracao_datas_simples_formatos_diferentes) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.013s

OK



Testando frase: 'Preciso dos documentos de 10 de outubro de 2022'
Datas extraídas: ['2022-10-10']

Testando frase: 'Verifique os registros de 05/12/2021'
Datas extraídas: ['2021-12-05']

Testando frase: 'Quero informações das normas do ano passado'
Datas extraídas: [('2023-01-01', '2023-12-31')]

Testando frase: 'Veja os dados de 15/07/19'
Datas extraídas: ['2019-07-15']

Testando frase: 'Os relatórios lançados há 5 dias'
Datas extraídas: ['2024-08-25']


### Teste 10 - Extração de Datas Inexistentes ou Mal Formadas:

In [43]:
class TesteExtracaoDatasInexistentesOuMalFormadas(unittest.TestCase):

    # Configuração inicial
    def setUp(self):
      self.casos_teste = [
        {
          "frase": "Relatórios de 32/13/2022",
          "datas_esperadas": None
        },
        {
          "frase": "Encontro marcado para 31 de Fevereiro de 2021",
            "datas_esperadas": None
        },
        {
          "frase": "Verifique os dados de 99/99/9999",
          "datas_esperadas": None
        },
        {
          "frase": "As datas 00/00/0000 não são válidas",
          "datas_esperadas": None
        },
        {
          "frase": "Preciso dos documentos de 29 de fevereiro de 2019",
          "datas_esperadas": None  # 2019 não é um ano bissexto
        }
      ]

    # Teste de extração de datas inexistentes ou mal formadas
    def teste_extracao_datas_inexistentes_mal_formadas(self):
        for teste in self.casos_teste:
          frase = teste["frase"]
          datas_esperadas = teste["datas_esperadas"]

          try:
            # Executar a função de extração de datas
            print(f"\nTestando frase: '{frase}'")
            datas_extraidas = extract_dates(frase)

            # Se datas_extraidas é None, fornecer mais detalhes para depuração
            if datas_extraidas is None:
              print(f"Correto: Nenhuma data extraída para a frase: '{frase}'.")
            else:
              print(f"Erro: Datas extraídas para uma entrada inválida: '{frase}'. Extraído: {datas_extraidas}")

            # Verificar se a saída é None para entradas inválidas
            self.assertIsNone(datas_extraidas,
                              f"Erro na extração de datas para a frase: '{frase}'. Esperado: None, Encontrado: {datas_extraidas}")

          except AssertionError as e:
              print(f"Falha no teste para a frase: '{frase}'. {str(e)}")
              raise e
          except Exception as e:
              print(f"Erro inesperado ao testar a frase: '{frase}'. {str(e)}")
              raise e

roda_teste_especifico(TesteExtracaoDatasInexistentesOuMalFormadas)

teste_extracao_datas_inexistentes_mal_formadas (__main__.TesteExtracaoDatasInexistentesOuMalFormadas.teste_extracao_datas_inexistentes_mal_formadas) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.013s

OK



Testando frase: 'Relatórios de 32/13/2022'
Correto: Nenhuma data extraída para a frase: 'Relatórios de 32/13/2022'.

Testando frase: 'Encontro marcado para 31 de Fevereiro de 2021'
Correto: Nenhuma data extraída para a frase: 'Encontro marcado para 31 de Fevereiro de 2021'.

Testando frase: 'Verifique os dados de 99/99/9999'
Correto: Nenhuma data extraída para a frase: 'Verifique os dados de 99/99/9999'.

Testando frase: 'As datas 00/00/0000 não são válidas'
Correto: Nenhuma data extraída para a frase: 'As datas 00/00/0000 não são válidas'.

Testando frase: 'Preciso dos documentos de 29 de fevereiro de 2019'
Correto: Nenhuma data extraída para a frase: 'Preciso dos documentos de 29 de fevereiro de 2019'.


### Teste 11 - Extração de Datas Semânticas e Múltiplas:

In [44]:
import unittest
from datetime import datetime, timedelta

class TesteExtracaoDatasSemanticasEMultiplas(unittest.TestCase):

    # Configuração inicial
    def setUp(self):
        # Data atual considerada para o teste
        self.data_atual = datetime.strptime("2024-08-30", "%Y-%m-%d")
        self.casos_teste = [
            {
                "frase": "Preciso dos dados de hoje e de ontem",
                "datas_esperadas": [
                    self.data_atual.strftime('%Y-%m-%d'),
                    (self.data_atual - timedelta(days=1)).strftime('%Y-%m-%d')
                ]
            },
            {
                "frase": "Quero os relatórios de anteontem, ontem e hoje",
                "datas_esperadas": [
                    (self.data_atual - timedelta(days=2)).strftime('%Y-%m-%d'),
                    (self.data_atual - timedelta(days=1)).strftime('%Y-%m-%d'),
                    self.data_atual.strftime('%Y-%m-%d')
                ]
            },
            {
                "frase": "Verifique os documentos de 10 de março de 2022 e 15 de maio de 2022",
                "datas_esperadas": ['2022-03-10', '2022-05-15']
            },
        ]

    # Teste de extração de múltiplas datas semânticas
    def teste_extracao_datas_semanticas_multiplas(self):
        for teste in self.casos_teste:
            frase = teste["frase"]
            datas_esperadas = teste["datas_esperadas"]

            try:
                # Executar a função de extração de datas
                print(f"\nTestando frase: '{frase}'")
                datas_extraidas = extract_dates(frase)

                # Se datas_extraidas é None ou incompleta, fornecer mais detalhes para depuração
                if datas_extraidas is None or len(datas_extraidas) != len(datas_esperadas):
                    print(f"Erro: Extração de datas incompleta ou incorreta para a frase: '{frase}'.")
                    print(f"Esperado: {datas_esperadas}, Encontrado: {datas_extraidas}")
                else:
                    print(f"Datas extraídas: {datas_extraidas}")

                # Verificar se todas as datas extraídas correspondem às datas esperadas
                self.assertEqual(datas_extraidas, datas_esperadas,
                                 f"Erro na extração de múltiplas datas para a frase: '{frase}'. Esperado: {datas_esperadas}, Encontrado: {datas_extraidas}")

            except AssertionError as e:
                print(f"Falha no teste para a frase: '{frase}'. {str(e)}")
                raise e
            except Exception as e:
                print(f"Erro inesperado ao testar a frase: '{frase}'. {str(e)}")
                raise e

# Executa o teste específico
roda_teste_especifico(TesteExtracaoDatasSemanticasEMultiplas)


teste_extracao_datas_semanticas_multiplas (__main__.TesteExtracaoDatasSemanticasEMultiplas.teste_extracao_datas_semanticas_multiplas) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.014s

OK



Testando frase: 'Preciso dos dados de hoje e de ontem'
Datas extraídas: ['2024-08-30', '2024-08-29']

Testando frase: 'Quero os relatórios de anteontem, ontem e hoje'
Datas extraídas: ['2024-08-28', '2024-08-29', '2024-08-30']

Testando frase: 'Verifique os documentos de 10 de março de 2022 e 15 de maio de 2022'
Datas extraídas: ['2022-03-10', '2022-05-15']
